<a href="https://colab.research.google.com/github/archyyu/encoder-related/blob/main/encoder_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
I think this is a very simple Masked Language Model

In [1]:
import requests
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import math
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/archyyu/publicResource/main/chat_dataset.csv')

In [3]:
# Hyperparameters
hidden_size = 100
embedding_dim = 80
seq_length = 25
learning_rate = 1e-1
batch_size = 20
dropout = 0.1
eval_iters = 200
num_heads = 4
head_size = 20

In [4]:
pad = '[pad]'
mask = '[MASK]'
data = []
targets = []
for index, row in df.iterrows():
  data.append(row['message'])
  targets.append(row['sentiment'])

datalen = []
for line in data:
  datalen.append(len(line.split(' ')))


targetset = sorted(set(targets))
sentiment_to_index = {s:i for i, s in enumerate(targetset)}
index_to_sentiment = {i:s for i, s in enumerate(targetset)}

In [5]:
dataset = sorted(set((' '.join(data)).split(' ')))
dataset.append(pad)
dataset.append(mask)
vocab_size = len(dataset)
word_to_index = {w:i for i, w in enumerate(dataset)}
index_to_word = {i:w for i, w in enumerate(dataset)}

pad_index = word_to_index[pad]

n = (int)(len(data) * 0.9)
training_data = data[:n]
val_data = data[n:]

In [6]:
lines = []
for item in data:
  lines.append(item.split(' '))

max_line = max([len(line) for line in lines])

for item in lines:
  for _ in range(max_line - len(item)):
    item.append(pad)

X = []
for line in lines:
  item = [word_to_index[word] for word in line]
  X.append(item)

Y = []
for i in range(len(targets)):
  item = sentiment_to_index[targets[i]]
  Y.append(item)

In [21]:
class AttentionHead(nn.Module):
  def __init__(self, embedding_size, head_size):
    super(AttentionHead, self).__init__()
    self.head_size = head_size
    self.C = embedding_size

    self.q = nn.Linear(self.C, head_size, bias=False)
    self.v = nn.Linear(self.C, head_size, bias=False)
    self.k = nn.Linear(self.C, head_size, bias=False)

  def forward(self, x, mask):
    B,T,C = x.shape
    q = self.q(x)
    k = self.k(x)
    v = self.v(x)

    wei = q @ k.transpose(-2, -1) * (self.head_size ** -0.5)
    wei.masked_fill_(mask==0, -1e9)
    wei = F.softmax(wei, dim=-1)

    return wei @ v

class EncoderMultiHeadAttention(nn.Module):
  def __init__(self, num_heads, embedding_size, head_size):
    super(EncoderMultiHeadAttention, self).__init__()
    self.num_heads = num_heads

    self.heads = nn.ModuleList([
        AttentionHead(embedding_size, head_size) for _ in range(num_heads)
    ])

    self.final_linear = nn.Linear(num_heads * head_size, embedding_size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask):
    head_outputs = [head(x, mask) for head in self.heads]
    concatenated_output = torch.cat(head_outputs, dim=-1)
    final_output = self.final_linear(concatenated_output)
    return self.dropout(final_output)

class FeedForward(nn.Module):
  def __init__(self, embedding_size):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(embedding_size, 4 * embedding_size),
        nn.ReLU(),
        nn.Linear(4 * embedding_size, embedding_size),
        nn.Dropout(dropout),
    )
  def forward(self, x):
    return self.net(x)

class AddAndNormLayer(nn.Module):
  def __init__(self, embedding_size):
    super(AddAndNormLayer, self).__init__()
    self.norm = nn.LayerNorm(embedding_size)

  def forward(self, x, subLayer: nn.Module, mask=None):
    if mask == None:
      return x + subLayer(self.norm(x))
    else:
      return x + subLayer(self.norm(x), mask)

class EncoderBlockAttention(nn.Module):
  def __init__(self, num_heads, embedding_size, head_size):
    super(EncoderBlockAttention, self).__init__()
    self.multiheads = EncoderMultiHeadAttention(num_heads, embedding_size, head_size)
    self.fw = FeedForward(embedding_size)
    self.addLayers = nn.ModuleList([AddAndNormLayer(embedding_size) for i in range(2)])

  def forward(self, x, mask):
    inter_result = self.addLayers[0](x, self.multiheads, mask)
    final_result = self.addLayers[1](inter_result, self.fw)
    return final_result

class Encoder(nn.Module):
  def __init__(self, num_heads, vocab_size, embedding_size, head_size):
    super(Encoder, self).__init__()
    self.em = nn.Embedding(vocab_size, embedding_size)
    self.embedding_size = embedding_size
    self.blocks = nn.ModuleList([EncoderBlockAttention(num_heads, embedding_size, head_size) for _ in range(4)])
    self.f_norm = nn.LayerNorm(embedding_size)
    self.fw = nn.Linear(embedding_size, vocab_size, bias=False)

  def p_en(self, x, embedding_size):
    B,T = x.shape
    C = embedding_size
    n = 10000
    x = torch.zeros((T, C))
    for k in range(T):
      for i in torch.arange(int(C/2)):
        denominator = torch.pow(n, 2*i/C)
        x[k, 2*i] = torch.sin(k/denominator)
        x[k, 2*i+1] = torch.cos(k/denominator)
    return x

  def forward(self, x, mask):
    B,T = x.shape
    x_em = self.em(x)
    p_em = self.p_en(x, self.embedding_size)

    x = x_em + p_em
    for block in self.blocks:
      x = block(x, mask)
    x = self.f_norm(x)
    x = self.fw(x)
    return x

criterion = nn.CrossEntropyLoss()

model = Encoder(num_heads, vocab_size, embedding_dim, head_size)
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)

In [20]:
def get_batch():
  inputs = []
  targets = []
  mask_index_list = []
  masks = []
  n = torch.randint(len(X) - batch_size, [1]).item()
  for i in range(batch_size):
    inputs_item = torch.tensor(X[n + i])

    mask_index = torch.randint(datalen[n + 1], [1])

    targets.append(inputs_item[mask_index])
    mask_index_list.append(torch.tensor([i,mask_index]))

    inputs_item[mask_index] = word_to_index[mask]

    inputs.append(inputs_item)

    masks.append(inputs_item != pad_index)

  return torch.stack(inputs), torch.stack(targets), torch.stack(mask_index_list), torch.stack(masks).unsqueeze(1)

In [22]:
n_iters = 10000
for i in range(n_iters):
  inputs, targets, indices, masks = get_batch()

  predicts = model(inputs, masks)

  #print(targets.view(-1).shape)
  outputs = predicts[indices[:,0],indices[:,1]]
  #print(outputs.shape)

  optimizer.zero_grad(set_to_none=True)

  loss = criterion(outputs, targets.view(-1))

  loss.backward()

  optimizer.step()

  if i%500 == 0:
    print(f'i {i}, loss:{loss.item()}')

i 0, loss:7.051021575927734
i 500, loss:3.3548896312713623
i 1000, loss:1.8986141681671143
i 1500, loss:2.221907138824463
i 2000, loss:1.1237813234329224
i 2500, loss:1.2446720600128174
i 3000, loss:2.3592095375061035
i 3500, loss:1.0986623764038086
i 4000, loss:2.5190157890319824
i 4500, loss:0.896750807762146
i 5000, loss:3.631227970123291
i 5500, loss:2.441333055496216
i 6000, loss:0.6416040658950806
i 6500, loss:0.8256516456604004
i 7000, loss:4.587604999542236
i 7500, loss:0.5165543556213379
i 8000, loss:0.9975517988204956
i 8500, loss:0.9209945797920227
i 9000, loss:0.3710198998451233
i 9500, loss:1.0797696113586426


In [26]:
n = torch.randint(len(X) - 20,[1]).item()
for line in data[-n:-n + 15]:
  hgg = [word_to_index[word] for word in line.split(' ')]
  linelen = len(hgg)
  for i in range(max_line - len(hgg)):
    hgg.append(word_to_index[pad])
  maskindex = torch.randint(linelen,[1]).item()
  hgg[maskindex] = word_to_index['[MASK]']
  print(' '.join([index_to_word[item] for item in hgg]))
  hgg = torch.tensor(hgg).unsqueeze(0)
  mask = (hgg != pad_index)
  predict = model(hgg, mask)
  values, indexes = torch.topk(predict[0][maskindex],3)
  for item in indexes:
    print('  ',index_to_word[item.item()])


I'm neutral [MASK] this [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad]
   about
   on
   this
The food was [MASK] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad]
   satisfactory
   delicious
   outstanding
The service at this hotel was [MASK] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad]
   great
   [pad]
   hotel
This phone [MASK] standard [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad]
   is
   restaurant
   was
The [MASK] is normal [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad]
   traffic
   car
   weather
This movie was [MASK] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad] [pad]
   regular
   ordinary
   mediocre
I have no strong opinion [MASK] this [pad] [pad] [pad] [pad] [pad] [pa

that is amazing
I masked one word in the sentence, then training to model to predict the masked word

after some epoches, it works. even though, it maynot predicts the exact word, but it will predict very similar word

will continue

the same with encoder_sentiment
updated with three things
1: use a class to encapsulate the add and norm layer
2: mask the pad tokens, so the pad tokens did not get attention
3: add position encoding

overall, the performance is better now.